# Gender Recognition by Voice Using Numerical Algorithms

## Importing Data

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

sns.set(font_scale=1.5)

data = pd.read_csv('data/cleaned/voice_data.csv')
X_train = pd.read_csv('data/cleaned/X_train.csv')
y_train = pd.read_csv('data/cleaned/y_train.csv')
X_test = pd.read_csv('data/cleaned/X_test.csv')
y_test = pd.read_csv('data/cleaned/y_test.csv')

In [4]:
data.head()

,meanfreq,median,Q25,Q75,IQR,sd,log_skew,log_kurt,sp.ent,sfm,mode,meanfun,log_minfun,exp_maxfun,modindx,meandom,mindom,maxdom,dfrange,label
0,-4.049248,-4.224901,-2.576102,-5.693607,-0.214778,0.427355,2.925892,3.177486,-0.039083,0.471575,-2.141210,-1.812038,-1.384486,0.636793,-1.454772,-1.564205,-0.708404,-1.431422,-1.419137,male
1,-3.841053,-3.999293,-2.486885,-5.588987,-0.258485,0.611669,4.032721,4.022523,-0.065236,0.594431,-2.141210,-1.079594,-1.369352,-0.524133,-1.014103,-1.561916,-0.708404,-1.418107,-1.405818,male
2,-3.463066,-4.095851,-2.706986,-3.928699,0.909326,1.603848,4.675089,4.506253,-1.083730,0.398261,-2.141210,-1.365368,-1.390131,0.404082,-1.065344,-1.563866,-0.708404,-1.429203,-1.416917,male
3,-0.992157,-0.759454,-0.901418,-0.711205,0.632690,0.899998,-0.927599,-0.837709,1.516383,1.797340,-1.054576,-1.666966,-1.143909,-0.524133,0.614286,-1.195367,-0.708404,-1.273867,-1.261532,male
4,-1.530640,-1.676948,-1.268395,-0.792029,1.005588,1.322561,-1.055855,-0.807635,1.708336,2.114740,-0.790514,-1.127233,-1.239725,0.189238,0.289046,-0.221660,-0.708404,0.124154,0.136933,male


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   meanfreq    3168 non-null   float64
 1   median      3168 non-null   float64
 2   Q25         3168 non-null   float64
 3   Q75         3168 non-null   float64
 4   IQR         3168 non-null   float64
 5   sd          3168 non-null   float64
 6   log_skew    3168 non-null   float64
 7   log_kurt    3168 non-null   float64
 8   sp.ent      3168 non-null   float64
 9   sfm         3168 non-null   float64
 10  mode        3168 non-null   float64
 11  meanfun     3168 non-null   float64
 12  log_minfun  3168 non-null   float64
 13  exp_maxfun  3168 non-null   float64
 14  modindx     3168 non-null   float64
 15  meandom     3168 non-null   float64
 16  mindom      3168 non-null   float64
 17  maxdom      3168 non-null   float64
 18  dfrange     3168 non-null   float64
 19  label       3168 non-null  

The meaning of the features are as follows:

* `meanfreq`: mean frequency (in kHz)
* `median`: median frequency (in kHz)
* `Q25`: first quantile (in kHz)
* `Q75`: third quantile (in kHz)
* `IQR`: inter-quantile range (in kHz)
* `sd`: standard deviation of frequency
* `log_skew`: skewness after logarithmic transformation
* `log_kurt`: kurtosis after logarithmic transformation
* `sp.ent`: spectral entropy
* `sfm`: spectral flatness
* `mode`: mode frequency
* `log_meanfun`: average of fundamental frequency measured across acoustic signal
* `log_minfun`: minimum fundamental frequency measured across acoustic signal after logarithmic transformation
* `exp_maxfun`: maximum fundamental frequency measured across acoustic signal after exponential transformation
* `modindx`: modulation index
* `meandom`: average of dominant frequency measured across acoustic signal
* `mindom`: minimum of dominant frequency measured across acoustic signal
* `maxdom`: maximum of dominant frequency measured across acoustic signal
* `dfrange`: range of dominant frequency measured across acoustic signal
* `label`: male or female

In [14]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

class CombinedMethod(object):
    def __init__(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        self.gbClassifier = RandomForestClassifier(random_state=87).fit(X_train, y_train.squeeze())
        self.svmClassifier = SVC(random_state=87, kernel="rbf", gamma=0.1).fit(X_train, y_train.squeeze())
        self.mlpClassifier = MLPClassifier(random_state=87, max_iter=1000, hidden_layer_sizes=(100, 100, 100, 100)).fit(X_train, y_train.squeeze())

    def predict(self, X):
        gbPredictions = self.gbClassifier.predict(X)
        svmPredictions = self.svmClassifier.predict(X)
        mlpPredictions = self.mlpClassifier.predict(X)
        predictions = []
        for i in range(len(X)):
            predictions.append(np.argmax(np.bincount([gbPredictions[i], svmPredictions[i], mlpPredictions[i]])))
        return predictions

    def score(self, X, y):
        predictions = self.predict(X)
        return np.sum(predictions == y.squeeze()) / len(y)

    def scoreAll(self, X, y):
        gbScore = self.gbClassifier.score(X, y.squeeze())
        svmScore = self.svmClassifier.score(X, y.squeeze())
        mlpScore = self.mlpClassifier.score(X, y.squeeze())
        combinedScore = self.score(X, y)
        return gbScore, svmScore, mlpScore, combinedScore

In [15]:
y_train_clean = y_train == "male"
y_test_clean = y_test == "male"
combinedMethod = CombinedMethod(X_train, y_train_clean)
print("Combined Method Score: ", combinedMethod.score(X_test, y_test_clean))

Combined Method Score:  0.9826498422712934


In [16]:
combinedMethod.scoreAll(X_test, y_test_clean)

(0.9842271293375394,
 0.9826498422712934,
 0.9810725552050473,
 0.9826498422712934)